In [ ]:
import tabula
import pandas as pd
import re
import numpy as np
import math

In [ ]:
file_path = "Downloads/woo-unused-voucher-codes-2021-01-11.pdf"
page_num = 234
#page_num = 6

In [ ]:
cols = [ 'Voucher Code', 'Product Information', 'Buyers Information', 'Order Title', 'Order Information', 'Recipient Information']

df = pd.DataFrame([])
df_combine = pd.DataFrame([])
for pageiter in range(page_num):
#for pageiter in [0,1,232,233]:
    print(pageiter)
    df = tabula.read_pdf( file_path, pages = pageiter+1, guess=False)
    #If you want to change the table by editing the columns you can do that here.
    if pageiter+1 == 1:
        df = df[0]
        df.columns = df.iloc[0]
        df = df[1:].reset_index()
        #df_combine = pd.concat([ df_combine, df]) 
        df = df.drop('index', axis = 1)
        df.insert(3, 'Order Title', np.nan)
        df.columns = cols
        df_combine = pd.concat([ df_combine, df]) 
    elif pageiter +1 == page_num:
        df = df[0]
        #df = df.drop('Unnamed: 0', axis = 1)
        df.columns = cols
        df_combine = pd.concat([ df_combine, df]) 
    else:
        #print(df[0]))
        df[0].columns = cols
        df_combine = pd.concat([ df_combine, df[0]]) #again you can choose between merge or concat as per your need
    #if pageiter > 3:
        #break

In [ ]:
df_combine

In [ ]:
df_combine = df_combine.reset_index().drop('index', axis = 1)
df_combine

In [ ]:
for index, row in df_combine.iterrows():
    #print(row['Voucher Code'])
    if type(row['Voucher Code']) == str:
        print(index)
        current = row['Voucher Code']
        #df_combine['Voucher Code'][index] = current
    else:
        df_combine.at[index, 'Voucher Code'] = current

In [ ]:
prices2020 = df_combine[df_combine['Order Information'].str.contains('2020', na = False)]['Product Information']
#prices2020['Year'] = 2020
prices2019 = df_combine[df_combine['Order Information'].str.contains('2019', na = False)]['Product Information']
#prices2019['Year'] = 2019

prices = pd.concat([prices2020, prices2019])
#prices = prices2020
prices

In [ ]:
df_prices = pd.DataFrame()

for index, price in prices.iteritems():
    for count in range(0,15):
        index = index + count
        print(index)
        price = df_combine.iloc[index]['Product Information']        
        if isinstance(price, str):
            if 'Price:' in price:
                break

    price = float(price.split(" ")[-1].replace('.','').replace(',','.'))

    code = df_combine['Voucher Code'].iloc[index]
    
    sku = code.split("-")[0]
    
    df_new = pd.DataFrame([[code, sku, price]])
    df_prices = pd.concat([df_prices, df_new])

df_prices = df_prices.reset_index().drop('index', axis = 1)
df_prices.columns = ['Kode', 'Sku', 'Pris']
df_prices

In [ ]:
df_oppsummering = df_prices.groupby('Sku').sum().reset_index()
df_oppsummering.columns = ['Sku', 'Sum']
df_oppsummering
df_oppsummering = df_oppsummering.merge(df_prices.drop('Kode', axis = 1).drop_duplicates('Sku'), how = 'inner', on = 'Sku')
df_oppsummering.to_csv('unused_vouchers_2020-2019.csv', sep=";")

In [ ]:
df_oppsummering

In [ ]:
df_oppsummering.iloc[:-1]['Sum'].sum()